In [2]:
import pandas as pd
df=pd.read_csv("C:\\Users\\giovi\\Downloads\\finale.csv",delimiter=";")

In [3]:
df=df.drop("pdf",axis=1)

In [8]:
df.head()

,Principio Attivo,Descrizione Gruppo,Denominazione e Confezione,Titolare AIC,Codice AIC,Codice Gruppo Equivalenza,classe,Codice ATC,URL,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,6.2,risposta_llm
0,Abacavir,ABACAVIR 20MG/ML 240ML USO ORALE,ZIAGEN*orale soluz flacone 240 ml 20 mg/ml,VIIV HEALTHCARE BV,34499020,AAB,H,J05AF06 - ABACAVIR,https://medicinali.aifa.gov.it/it/#/it/dettagl...,Ziagen è indicato nella terapia di associazion...,Ziagen deve essere prescritto da medici espert...,Ipersensibilità ad abacavir o ad uno qualsiasi...,Reazioni di ipersensibilità (vedere anche para...,La possibilità che il P450 medi interazioni di...,"Gravidanza \n \nCome regola generale, quando s...",Non sono stati effettuati studi sulla capacità...,"Per molte delle reazioni avverse riportate, no...","Negli studi clinici, sono state somministrate ...",Non pertinente,
1,Abacavir,ABACAVIR 300MG 60 UNITA' USO ORALE,ABACAVIR*60 cpr riv 300 mg,MYLAN SpA,45354040,AAC,H,J05AF06 - ABACAVIR,https://medicinali.aifa.gov.it/it/#/it/dettagl...,Abacavir Mylan è indicato nella terapia di ass...,Posologia \nAbacavir Mylan deve essere prescri...,Ipersensibilità ad abacavir o ad uno qualsiasi...,Reazioni di ipersensibilità (vedere anche para...,Sulla base dei risultati degli esperimenti in ...,"Gravidanza \nCome regola generale, quando si d...",Non sono stati effettuati studi sulla capacità...,"Per molte delle reazioni avverse riportate, no...",Negli studi clinici sono state somministrate a...,Non pertinente.,La dimostrazione dell'utilità del farmaco si b...
2,Abacavir,ABACAVIR 300MG 60 UNITA' USO ORALE,ZIAGEN*60 cpr riv 300 mg,VIIV HEALTHCARE BV,34499018,AAC,H,J05AF06 - ABACAVIR,https://medicinali.aifa.gov.it/it/#/it/dettagl...,Ziagen è indicato nella terapia di associazion...,Ziagen deve essere prescritto da medici espert...,Ipersensibilità ad abacavir o ad uno qualsiasi...,Reazioni di ipersensibilità (vedere anche para...,La possibilità che il P450 medi interazioni di...,"Gravidanza \n \nCome regola generale, quando s...",Non sono stati effettuati studi sulla capacità...,"Per molte delle reazioni avverse riportate, no...","Negli studi clinici, sono state somministrate ...",Non pertinente,
3,Abacavir/lamivudina,ABACAVIR+LAMIVUDINA 600+300MG 30 UNITA' USO ORALE,ABACAVIR E LAMIVUDINA*30 cpr riv 600 mg + 300 mg,AUROBINDO PHARMA ITALIA Srl,44669012,AAD,H,J05AR02 - LAMIVUDINA E ABACAVIR,https://medicinali.aifa.gov.it/it/#/it/dettagl...,Abacavir e lamivudina Aurobindo è indicato nel...,La terapia deve essere prescritta da un medico...,Ipersensibilità ai principi attivi o ad uno qu...,In questo paragrafo vengono incluse le avverte...,Abacavir e lamivudina Aurobindo contiene abaca...,"Gravidanza \nCome regola generale, quando si d...",Non sono stati effettuati studi sulla capacità...,Riassunto del profilo di sicurezza \nLe reazio...,Non sono stati identificati sintomi e segni sp...,Non pertinente.,La dimostrazione dell'utilità del farmaco è da...
4,Abacavir/Lamivudina,ABACAVIR+LAMIVUDINA 600+300MG 30 UNITA' USO ORALE,ABACAVIR E LAMIVUDINA*30 cpr riv 600 mg + 300 mg,SUN PHARMACEUTICAL IND.EU.B.V.,45348036,AAD,H,J05AR02 - LAMIVUDINA E ABACAVIR,https://medicinali.aifa.gov.it/it/#/it/dettagl...,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,NOT AVAILABLE,La dimostrazione dell'utilità del farmaco è ba...


The LLM here queries each row on the dataset (Very Slow)

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import time

def query_ollama(prompt, timeout=180):
    try:
        response = requests.post(
            'http://localhost:11434/api/generate',
            json={
                "model": "gemma:2b",
                "prompt": prompt,
                "stream": False
            },
            timeout=timeout
        )
        return response.json()["response"]
    except Exception as e:
        return f"ERRORE: {type(e).__name__} - {str(e)}"

def process_row(index, testo):
    prompt = f"""
Il seguente testo è stato estratto da un database:

{testo}

Rispondi alla domanda: "Su cosa si basa la dimostrazione dell'utilità del farmaco?"

Risposta concisa e strutturata:
"""
    risposta = query_ollama(prompt)
    print(f"✓ Riga {index}: {risposta[:80]}...")
    return index, risposta

df = df[df["4.1"].notnull() & (df["4.1"].str.strip() != "")]
df = df.copy() 

df["risposta_llm"] = ""  

batch_size = 10
max_workers = 2

for batch_start in range(0, len(df), batch_size):
    batch_end = min(batch_start + batch_size, len(df))
    batch_data = [(i, df.iloc[i]["4.1"]) for i in range(batch_start, batch_end)]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_row, i, testo): i for i, testo in batch_data}
        for future in as_completed(futures):
            i = futures[future]
            try:
                index, result = future.result()
                df.iloc[index, df.columns.get_loc("risposta_llm")] = result
            except Exception as e:
                print(f"Errore elaborazione riga {i}: {e}")
                df.iloc[index, df.columns.get_loc("risposta_llm")] = "ERRORE"

    df.to_csv("risultati_parziali.csv", index=False)
    print(f"Batch {batch_start}-{batch_end} salvato")


df.to_csv("risultati_con_risposte.csv", index=False)


New, faster implementation: the code first prompts us to select a filter based on the type of drug. We can choose which column to use to filter the rows.

In [ ]:
import requests
import pandas as pd

def query_ollama(prompt, timeout=180):
    try:
        response = requests.post(
            'http://localhost:11434/api/generate',
            json={
                "model": "gemma:2b",
                "prompt": prompt,
                "stream": False
            },
            timeout=timeout
        )
        return response.json()["response"]
    except Exception as e:
        return f"ERRORE: {type(e).__name__} - {str(e)}"

def interroga_righe_con_parola(parola, df, domanda, colonna_cerca="Denominazione e Confezione", colonna_testo="4.1"):
    # Filtro righe che contengono la parola (case-insensitive)
    filtro = df[colonna_cerca].str.contains(parola, case=False, na=False)
    df_filtrato = df[filtro].copy()

    if df_filtrato.empty:
        print(f"Nessuna riga trovata contenente '{parola}' nella colonna '{colonna_cerca}'")
        return pd.DataFrame()

    # Nome della colonna per la risposta = domanda
    nome_colonna_risposta = domanda.strip().replace("?", "").strip()
    
    df_filtrato[nome_colonna_risposta] = ""

    for idx, row in df_filtrato.iterrows():
        testo = row[colonna_testo]
        prompt = f"""
Il seguente testo è stato estratto da un database:

{testo}

Rispondi alla domanda: "{domanda}"

Risposta concisa e strutturata:
"""
        risposta = query_ollama(prompt)
        df_filtrato.at[idx, nome_colonna_risposta] = risposta

        # Stampa a video
        print(f"Riga {idx}:")
        print(row.iloc[:8].to_string())
        print(f"{nome_colonna_risposta}: {risposta}\n{'-'*40}")

    # Seleziona solo le prime 8 colonne + colonna_testo + risposta
    colonne_output = list(df.columns[:8]) + [colonna_testo, nome_colonna_risposta]
    return df_filtrato[colonne_output]

if __name__ == "__main__":
    parola_cercata = input("Inserisci la parola da cercare in 'Denominazione e Confezione': ").strip()
    domanda = "Su cosa si basa la dimostrazione dell'utilità del farmaco?"

    df_risultati = interroga_righe_con_parola(parola_cercata, df, domanda)

    if not df_risultati.empty:
        df_risultati.to_csv("risultati_filtrati.csv", index=False)


Riga 1:
Principio Attivo                                        Abacavir
Descrizione Gruppo            ABACAVIR 300MG 60 UNITA' USO ORALE
Denominazione e Confezione            ABACAVIR*60 cpr riv 300 mg
Titolare AIC                                           MYLAN SpA
Codice AIC                                              45354040
Codice Gruppo Equivalenza                                    AAC
classe                                                         H
Codice ATC                                    J05AF06 - ABACAVIR
Su cosa si basa la dimostrazione dell'utilità del farmaco: La dimostrazione dell'utilità del farmaco basa sui risultati degli studi effettuati con un regime posologico di due volte al giorno, in pazienti adulti mai sottoposti a trattamento (naïve) in terapia di associazione (vedere paragrafo 5.1).
----------------------------------------
Riga 3:
Principio Attivo                                            Abacavir/lamivudina
Descrizione Gruppo            ABACAVIR+LAMIV

In [23]:
report=pd.read_csv("C:/Users/giovi/Desktop/ProgrammingProject/FarmaciDB/risultati_filtrati.csv")

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)  # mostra testo completo senza puntini
pd.set_option('display.expand_frame_repr', False)

report


,Principio Attivo,Descrizione Gruppo,Denominazione e Confezione,Titolare AIC,Codice AIC,Codice Gruppo Equivalenza,classe,Codice ATC,4.1,Su cosa si basa la dimostrazione dell'utilità del farmaco
0,Abacavir,ABACAVIR 300MG 60 UNITA' USO ORALE,ABACAVIR*60 cpr riv 300 mg,MYLAN SpA,45354040,AAC,H,J05AF06 - ABACAVIR,"Abacavir Mylan è indicato nella terapia di associazione antiretrovirale per il trattamento \ndell’infezione da Virus dell’Immunodeficienza Umana (HIV) negli adulti, negli adolescenti e nei \nbambini (vedere paragrafi 4.4 e 5.1). \n \nLa dimostrazione del beneficio di abacavir è principalmente basata sui risultati degli studi \neffettuati con un regime posologico di due volte al giorno, in pazienti adulti mai sottoposti a \ntrattamento (naïve) in terapia di associazione (vedere paragrafo 5.1). \n \nPrima di iniziare il trattamento con abacavir, deve essere eseguito uno screening per \ndeterminare la presenza dell’allele HLA-B*5701 in ogni paziente affetto da HIV, a prescindere \ndalla razza (vedere paragrafo 4.4). Abacavir non deve essere utilizzato nei pazienti in cui sia \nnota la presenza dell’allele \nHLA-B*5701.","La dimostrazione dell'utilità del farmaco basa sui risultati degli studi effettuati con un regime posologico di due volte al giorno, in pazienti adulti mai sottoposti a trattamento (naïve) in terapia di associazione (vedere paragrafo 5.1)."
1,Abacavir/lamivudina,ABACAVIR+LAMIVUDINA 600+300MG 30 UNITA' USO ORALE,ABACAVIR E LAMIVUDINA*30 cpr riv 600 mg + 300 mg,AUROBINDO PHARMA ITALIA Srl,44669012,AAD,H,J05AR02 - LAMIVUDINA E ABACAVIR,"Abacavir e lamivudina Aurobindo è indicato nella terapia di combinazione antiretrovirale per il \ntrattamento di adulti, adolescenti e bambini che pesano almeno 25 kg con infezione da Virus \ndell’Immunodeficienza Umana (Human Immunodeficiency Virus, HIV) (vedere paragrafi 4.4 e 5.1). \n \nPrima di iniziare il trattamento con abacavir, deve essere eseguito uno screening per la presenza \ndell’allele HLA-B*5701 in ogni paziente affetto da HIV, a prescindere dalla razza (vedere paragrafo \n4.4). Abacavir non deve essere utilizzato nei pazienti in cui sia nota la presenza dell’allele HLA-\nB*5701.","La dimostrazione dell'utilità del farmaco è fornita dal testo che descrive la terapia di combinazione antiretrovirale per il trattamento di adulti, adolescenti e bambini che pesano almeno 25 kg con infezione da Virus dell'Immunodeficienza Umana (Human Immunodeficiency Virus, HIV)."
2,Abacavir/Lamivudina,ABACAVIR+LAMIVUDINA 600+300MG 30 UNITA' USO ORALE,ABACAVIR E LAMIVUDINA*30 cpr riv 600 mg + 300 mg,SUN PHARMACEUTICAL IND.EU.B.V.,45348036,AAD,H,J05AR02 - LAMIVUDINA E ABACAVIR,NOT AVAILABLE,"Non ho accesso il testo, quindi non posso fornire una risposta alla tua domanda."
3,Abacavir/lamivudina/zidovudina,ABACAVIR+LAMIVUDINA+ZIDOVUDINA 300+150+300MG 60 UNITA' USO ORALE,ABACAVIR LAMIVUDINA ZIDOVUDINA*60x1 cpr riv 300 mg + 150 mg + 300 mg,MYLAN SpA,42645046,AAE,H,"J05AR04 - ZIDOVUDINA, LAMIVUDINA E ABACAVIR","Abacavir/Lamivudina/Zidovudina Mylan è indicato per il trattamento dell’infezione da Virus \ndell'Immunodeficienza Umana (Human Immunodeficiency Virus, HIV) in soggetti adulti (vedere \nparagrafi 4.4 e 5.1). Questa combinazione fissa sostituisce i tre componenti (abacavir, lamivudina e \nzidovudina) usati separatamente alle medesime dosi. Si raccomanda che il trattamento venga iniziato \ncon abacavir, lamivudina e zidovudina separatamente per le prime 6-8 settimane (vedere paragrafo 4.4). \nLa scelta di questa combinazione fissa dovrebbe essere basata non solo sul criterio di potenziale \naderenza ma prevalentemente sull'efficacia attesa e sui rischi correlati ai tre analoghi nucleosidi. \n \nLa dimostrazione del beneficio di abacavir/lamivudina/zidovudina si basa soprattutto sui risultati degli \nstudi effettuati nel trattamento di pazienti mai trattati o moderatamente trattati con antiretrovirali, con \nmalattia non avanzata. In pazienti con alta carica virale (>100.000 copie/ml) l